# Crown Analysis

This notebook provides code to analyse and explore point cloud tree crowns. As input you can either use an already separated crown point cloud or use a complete tree and use the provided separation code (see `option B` in step 1)

---------------

#### Imports

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import os
import numpy as np
import open3d as o3d

from utils import (
      ahn_utils,
      las_utils,
      tree_utils,
      o3d_utils
  )
from utils.interpolation import FastGridInterpolator

from labels import Labels
from config import Paths

In [ ]:
# Setup logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### 1. Load Data
---

**Option A**: Load crown point cloud

In [ ]:
las_file = '../../dataset/sonarski/crown_121913_487434.las'
crown_cloud = o3d_utils.read_las(las_file)

**Option B**: Load tree point cloud and separate using `src/utils/tree_utils.py`

In [ ]:
adTree_exe = Paths.get_adtree()

# Load point cloud data
las_file = '../../dataset/cyclo/filtered_tree_121913_487434.las'
tree_cloud = o3d_utils.read_las(las_file)

# Separate crown from stem
stem_cloud, crown_cloud = tree_utils.tree_separate(tree_cloud, adTree_exe)

### 2. Crown Analysis
---

**Corwn Dimensions**

In [ ]:
# estimate crown center
crown_center = crown_cloud.get_center()[:2]

# estimate crown height
crown_height = tree_utils.crown_height(crown_cloud)
print(f"Crown height is {crown_height:.2f} m")

# estimate crown diameter
crown_diameter = tree_utils.crown_diameter(crown_cloud)
print(f"Crown diameter is {crown_diameter:.2f} m")

**Crown distance to surface**

In [ ]:
# Load AHN Surface data
ahn_data_folder = '../../dataset/ahn_surf/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder)
treecode = las_utils.get_treecode_from_filename(las_file)
ahn_tile = npz_reader.filter_file(treecode)

# Estimate ahn_z
fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'],
                                    ahn_tile['ground_surface'])
tree_ahn_z = fast_z(crown_center.reshape(1,2))[0]


crown_base_height = tree_utils.crown_base_height(crown_cloud, tree_ahn_z)
print(f"Crown distance to surface is {crown_base_height:.2f} m")

**Crown Shape**

In [ ]:
# estimate crown shape (conical, inverse-conical, spherical, or cylindrical)
crown_shape = tree_utils.crown_shape(crown_cloud)
print(f"Crown shape is `{crown_shape}`")

**3D Crown Modelling**

The _method_ parameter in `crown_to_mesh()` can take the following values:
- _'convex_hull'_
- _'alphashape'_ (slow, but more accurate)

In [ ]:
# 3D crown modelling using 'convex_hull'
crown_mesh_hull, volume = tree_utils.crown_to_mesh(crown_cloud, method='convex_hull')
print(f"Crown volume is {crown_mesh_hull.get_volume():.2f} m3")
o3d_utils.plot_mesh(crown_mesh_hull)

In [ ]:
# 3D crown modelling using 'alphashape'
crown_mesh_alpha, volume = tree_utils.crown_to_mesh(crown_cloud, method='alphashape')
print(f"Crown volume is {crown_mesh_alpha.get_volume():.2f} m3")
o3d_utils.plot_mesh(crown_mesh_alpha)

In [ ]:
# interactive show mesh with cloud
o3d_utils.show_mesh_cloud(crown_mesh_hull, crown_cloud)

![convex_hull.png](../../imgs/crown_mesh_comparison.png)